In [ ]:
import numpy as np
import pandas as pd
import unittest
from IPython.display import display
import scipy.stats as stats
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Assign raw data and plate diagram files from environment

RAW_SAMPLE_1 = "../data/maya_test/raw_counts_maya.xlsx"
RAW_SAMPLE_1_DIAGRAM = "../data/maya_test/diagram_maya.xlsx"

# Read in the raw data and plate diagram as pandas dataframes
df_data = pd.read_excel(RAW_SAMPLE_1, sheet_name=0)
df_diagram = pd.read_excel(RAW_SAMPLE_1_DIAGRAM)


# Test the raw data and plate diagram to make sure they are the correct size, should be 96 rows by 16 columns and 8 rows by 13 columns respectively always
class TestStringMethods(unittest.TestCase):

    def test_raw_data(self):
        self.assertEqual(df_data.shape, (96, 15), f"Raw data is {df_data.shape[0]} rows by {df_data.shape[1]} columns but should be 96 rows by 16 columns")

    def test_plate_diagram(self):
        self.assertEqual(df_diagram.shape, (8, 13), f"Plate diagram is {df_diagram.shape[0]} rows by {df_diagram.shape[1]} columns but should be 8 rows by 13 columns")


unittest.main(argv=['first-arg-is-ignored'], exit=False)

display(df_data)
display(df_diagram)

In [ ]:
# Set the first column as the index, remove whitespace and add a space to the "dup" values
df_diagram = df_diagram.set_index(df_diagram.columns[0])
df_diagram = df_diagram.replace('\s+', '', regex=True)
df_diagram = df_diagram.replace(r'(?i)dup', ' dup', regex=True)


# Create an empty sample map dictionary
sample_map = {}

for row in df_diagram.index:
    for col in df_diagram.columns[1:]:
        well_id = f"{row}{int(col):02d}"
        sample_name = df_diagram.loc[row, col]
        sample_map[well_id] = sample_name

    """Use rows and columns (besides the first one) to relate the well ID to the sample name.

    For example, the first row name is A and the first column name is 1.
    The well ID will be A01 and the sample name will the sample in that row/column.

    """

# Read in the raw qPCR data and map the well IDs to sample names using the dictionary
df_data["Sample"] = df_data["Well"].map(sample_map)

# Show Well, Sample, and Cq columns
df_data[['Well', 'Sample', 'Cq']]

In [ ]:
# Select relevant columns (Well, Cq, and Sample)
df = df_data[['Well', 'Cq', 'Sample']].copy()

# Create mtDNA1 and mtDNA2 columns
df['mtDNA1'] = "mtDNA1"
df['mtDNA2'] = "mtDNA2"

# Arrange columns like this: "Well", "Sample", "mtDNA1", "mtDNA2", "Cq"
df = df.loc[:,["Well", "Sample", "mtDNA1", "mtDNA2", "Cq"]]

# Drop rows with NA values
df = df.dropna()

# Show first 5 rows
df

In [ ]:
# set mtDNA1 and mtDNA2 values to Cq values by treating mtDNA1 as the Cq for the first sample and mtDNA2 as the Cq for the duplicate sample if it exists as "Sample dup"

# Note, exactly "Sample dup" is used to avoid matching "Sample dup **" or any additions to the name

for row, index in df.iterrows():
    df.loc[row, 'mtDNA1'] = df.loc[row, 'Cq']
    if df.loc[row, 'Sample'] + ' dup' in df['Sample'].values:
        df.loc[row, 'mtDNA2'] = df.loc[df['Sample'] == df.loc[row, 'Sample'] + ' dup', 'Cq'].values[0]
    else:
        df.loc[row, 'mtDNA2'] = np.NAN

""" Assign mtDNA1 and mtNDA2 value

    For each row, set the mtDNA1 value to the Cq value and set the mtDNA2 value to the Cq value of the duplicate sample if it exists.

    For example, if the sample name is "D12", set the mtDNA1 value to its Cq value and set the mtDNA2 value to the Cq value of "D12 Dup" (if it exists).

    If the duplicate ("D12 Dup") does not exist, set the mtDNA2 value to NaN, which is then dropped later.
"""


# Drop the Cq column and drop NA values
df = df.drop(columns=['Cq'])
df = df.dropna()

# calculate standard deviation of each row
df['St.Dev'] = df[['mtDNA1', 'mtDNA2']].std(axis=1)

# Show first 5 rows
df

In [ ]:
# Throw warnings for standard deviations greater than .22

for row, index in df.iterrows():
    if df.loc[row, 'St.Dev'] > .22:
        print(f"\n Warning: Standard deviation for {df.loc[row, 'Sample']} is {round(df.loc[row, 'St.Dev'],ndigits=3)} "
              f"(Sample 1: {round(df.loc[row, 'mtDNA1'],ndigits=3)} vs Sample 2: {round(df.loc[row, 'mtDNA2'], ndigits=2)}) \n")

In [ ]:
#Drop index, sort by standard deviation (descending), and download the file
df = df.sort_values(by=['St.Dev'], ascending=False)
df.rename(columns = {'Sample' : 'Sample ID'}, inplace = True)
df = df.reset_index(drop=True)

df
# Uncomment to download the file
# df.to_excel("50_gcr_random_name_test_output.xlsx",
#           index=False)

In [ ]:
REFERENCE_FILE = '../data/maya_test/maya_reference_file.xlsx'

summary_file = pd.read_excel(REFERENCE_FILE, engine='openpyxl')

# join the summary file with the data file on the sample name

summary_file = summary_file.merge(df.loc[:, ('Sample ID', 'mtDNA1', 'mtDNA2')], on='Sample ID', how='left')

summary_file.rename(columns = {'mtDNA1' : 'Telomere 1', 'mtDNA2' : 'Telomere 2'}, inplace = True)

SRC_test_1 = [29.24,29.45,29.5,29.66,29.44,29.45,29.55,29.05,29.18,29.27,29.21,29.17,29.05,29.25,30.11,29.52,29.54,29.3,29.54,29.38,29.71,29.46,29.17,29.05,29.11,29.35,29.16,29.12,29.06,29.75,29.42,29.42,29.19,29.21,29.27,29.35
]

SRC_test_2 = [29.07, 29.41,29.5,29.38,29.73,29.59,29.63,29.08,29.18, 29.3,29.32,29.7,29,29.21,29.79,29.49,29.43,29.23,29.31,29.37,29.54,29.49,29.27,29.07,29.28,29.51,29.22,29.14,29.12,29.65,29.16,29.44,29.19,29.35,29.42,29.56]

summary_file['SCR 1'] = SRC_test_1
summary_file['SCR 2'] = SRC_test_2

# check for NaN values and return string difference when not mapped
for row, index in summary_file.iterrows():
    if np.isnan(summary_file.loc[row, 'Telomere 1']):
        print(f"{summary_file.loc[row, 'Sample ID']} not found in data file, deleting row")
        summary_file.drop(index=row, axis=0, inplace=True)

display(summary_file)

In [ ]:
summary_file['Avg Cq per sample, Telomere'] = summary_file.loc[:, ['Telomere 1', 'Telomere 2']].mean(axis=1)
summary_file['Avg Cq per sample, SCR'] = summary_file.loc[:, ['SCR 1', 'SCR 2']].mean(axis=1)

Avg_sq_per_target_telomere = summary_file['Avg Cq per sample, Telomere'].mean()
Avg_sq_per_target_SCR = summary_file['Avg Cq per sample, SCR'].mean()

summary_file['cCq per sample per target, Telomere'] = Avg_sq_per_target_telomere - summary_file['Avg Cq per sample, Telomere']

summary_file['cCq per sample per target, SCR'] = Avg_sq_per_target_SCR - summary_file['Avg Cq per sample, SCR']

summary_file['cCq Telomere - SCR'] = summary_file['cCq per sample per target, Telomere'] - summary_file['cCq per sample per target, SCR']

summary_file['RQ, 2^dCq Telomere'] = 2**summary_file['cCq per sample per target, Telomere']
summary_file['RQ, 2^dCq SCR'] = 2**summary_file['cCq per sample per target, SCR']

summary_file['Normalized Expression, RQ Telo/RQ SCR'] = summary_file['RQ, 2^dCq Telomere'] / summary_file['RQ, 2^dCq SCR']

display(summary_file)

In [ ]:
all_conditions = (summary_file['Full Tx'].unique())

# calculate geomean for each condition
sample_stats = []
for condition in all_conditions:
    condition_values = [sample for sample in summary_file.loc[summary_file['Full Tx'] == condition, 'Normalized Expression, RQ Telo/RQ SCR']]

    condition_geomean = stats.gmean(condition_values)

    sample_stats.append({'Condition': condition,
                         'St.Dev': np.std(condition_values),
                         'GeoMean': condition_geomean})

print("\n \n Geo Means:")
pprint(sample_stats)

In [ ]:
# plot a bar graph of each condition and their geomean

conditions_summary = pd.DataFrame(sample_stats)

sns.set_theme(style="white")
bar_plot = sns.barplot(x='Condition', y='GeoMean', data=conditions_summary)

for index, row in conditions_summary.iterrows():
    plt.errorbar(x=index, y=row['GeoMean'], yerr=row['St.Dev'], capsize=3, color='black', elinewidth=1, capthick=1)
    
for index, row in conditions_summary.iterrows():
    bar_plot.text(index, row['GeoMean'] + row['St.Dev'] - 0.5, round(row['GeoMean'], 2), color='black', ha='center', fontsize=10)

plt.tight_layout()
    
plt.show()